In [4]:
from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
import dspy
from dspy.teleprompt import BootstrapFewShot

from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
init_sys_env()
secret_env_file = '/mnt/d/project/dy/extra/autogen/env_secret_config.yaml'

api_configs = read_yaml(secret_env_file)


In [5]:
# Set up the LM
model_config = api_configs.get('openai')
turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=520,api_key=model_config.get('api_key'))
dspy.settings.configure(lm=turbo)

# Load math questions from the GSM8K dataset
gsm8k = GSM8K() # GSM8K 数据集的训练部分，包含用于训练模型解决数学问题的数据。通过使用这个训练集，语言模型可以学习并提高在数学推理和问题解决方面的能力。
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]
gsm8k_trainset
# gsm8k_trainset 和 gsm8k_devset 数据集包含一个示例列表，每个示例都有 question 和 answer 字段。



In [7]:
gsm8k_trainset[0]

In [3]:
# 环境设置完成后，让我们定义一个自定义程序，利用 ChainOfThought 模块执行逐步推理以生成答案：

class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)


In [4]:
from dspy.teleprompt import BootstrapFewShot

# 设置优化器：我们想要“引导”（即自生成）CoT 程序的 4 次示例。
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# 优化！在这里使用 `gsm8k_metric`。通常，度量标准会告诉优化器它的表现如何。
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset)


In [8]:
# 评估器（Evaluator）用于评估模型在特定数据集上的性能。通过设置评估器，可以多次评估模型，观察其在不同参数配置下的表现。
from dspy.evaluate import Evaluate

# evaluate = Evaluate(
#     devset=gsm8k_devset,    # 评估数据集
#     metric=gsm8k_metric,    # 评估指标
#     num_threads=4,          # 使用的线程数
#     display_progress=True,  # 是否显示进度
#     display_table=0         # 是否显示评估结果表格
# )
# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=gsm8k_devset, metric=gsm8k_metric, num_threads=8, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

In [9]:
# 在 DSPy 中，turbo.inspect_history(n=1) 用于查看最近一次语言模型的生成记录。这个功能可以帮助您了解模型在生成过程中使用的提示和输出，从而有助于调试和优化提示。
# 查看最近一次生成记录
turbo.inspect_history(n=1)


In [12]:
# Optimizers Evaluators

In [10]:
# 在 DSPy 中，通过提示器和自定义模块编译得到的 optimized_cot 程序可以直接用于生成答案。您可以向 optimized_cot 传递一个问题，并获取模型生成的答案。
# 使用 optimized_cot 程序回答问题
question = "What is the sum of 15 and 27?"
answer = optimized_cot(question=question)
answer
